In [1]:
import pandas as pd
import os
import glob
import numpy as np
import warnings

import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import re

warnings.filterwarnings('ignore')
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
my_dir='/content/drive/My Drive/MLDataSets/titanic/'
os.chdir( my_dir )

In [0]:
test_df = pd.read_csv(my_dir+"test.csv")


In [0]:
train_df = pd.read_csv(my_dir+"train.csv")
train1 = train.drop('Survived',axis=1)
df = pd.concat([train1,test_df])

In [38]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:
from pandas_profiling import ProfileReport


In [40]:
profile = ProfileReport(df)

profile

Number of variables,12
Number of observations,1309
Total Missing (%),8.1%
Total size in memory,122.8 KiB
Average record size in memory,96.1 B
Numeric,7
Categorical,5
Boolean,0
Date,0
Text (Unique),0
Rejected,0




Data Exploration Analysis




Feature Engineering
Selecting features and treating missing values

In [0]:
# Evaluation of the age variable
df['Age'] = df['Age'].fillna(method='ffill')
df['Age_bin'] = pd.qcut(df['Age'], 4, labels=False)
df['Age_bin']= df['Age_bin'].round(0).astype(str)

In [0]:
# Fare variable evaluation
df['Fare'] = df['Fare'].fillna(method = 'ffill')
df['Fare_bin'] = pd.qcut(df['Fare'], 4, labels=False)
df['Fare_bin']= df['Fare_bin'].round(0).astype(str)


In [0]:
# Evaluation of the Name variable (Title)
df['Title'] = [nameStr[1].strip().split('.')[0] for nameStr in df['Name'].str.split(',')]

In [0]:
# Family size
df['FamilySize'] =  df['SibSp'] + df['Parch'] + 1


In [0]:
# Evaluation of the Cabin variable
df['IsCabinDataEmpty'] = 0
df.loc[df['Cabin'].isnull(),'IsCabinDataEmpty'] = 1

In [0]:

# Evaluation of the PClass variable
df['Pclass'] = df['Pclass'].astype(str)

In [47]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_bin,Fare_bin,Title,FamilySize,IsCabinDataEmpty
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,0,Mr,2,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,3,Mrs,2,0
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1,Miss,1,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,3,Mrs,2,0
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,2,1,Mr,1,1


In [0]:
# Selection of variables
df2 = df.filter(['Pclass', 'Sex', 'Embarked', 'Age_bin', 'Fare_bin', 'Title', 'FamilySize', 'IsCabinDataEmpty'])

In [49]:
df2.head()

,Pclass,Sex,Embarked,Age_bin,Fare_bin,Title,FamilySize,IsCabinDataEmpty
0,3,male,S,1,0,Mr,2,1
1,1,female,C,2,3,Mrs,2,0
2,3,female,S,1,1,Miss,1,1
3,1,female,S,2,3,Mrs,2,0
4,3,male,S,2,1,Mr,1,1


In [50]:
profile = ProfileReport(df2)
profile

Number of variables,9
Number of observations,1309
Total Missing (%),0.0%
Total size in memory,92.2 KiB
Average record size in memory,72.1 B
Numeric,2
Categorical,6
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [0]:
# Transformation from categorical to numeric variables
df3 = pd.get_dummies(df2)

In [52]:
df3.head()

,FamilySize,IsCabinDataEmpty,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Age_bin_0,Age_bin_1,Age_bin_2,Age_bin_3,Fare_bin_0,Fare_bin_1,Fare_bin_2,Fare_bin_3,Title_Capt,Title_Col,Title_Don,Title_Dona,Title_Dr,Title_Jonkheer,Title_Lady,Title_Major,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess
0,2,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,2,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1,1,0,0,1,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,2,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,1,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [0]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,roc_curve,recall_score, classification_report,f1_score, precision_recall_fscore_support)


In [0]:
# Train and test dataset
X = df3.iloc [0: 891]
y = df3.iloc [891: 1309]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                        train_df['Survived'],
                                        test_size=0.2,
                                        random_state=67)

Selection of Variables


In [55]:
sel = SelectKBest(mutual_info_classif, k=15).fit(X_train,y_train)
selected_variables = list(X_train.columns[sel.get_support()])
print(selected_variables)

['FamilySize', 'IsCabinDataEmpty', 'Pclass_1', 'Pclass_3', 'Sex_female', 'Sex_male', 'Age_bin_0', 'Age_bin_3', 'Fare_bin_0', 'Fare_bin_3', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_the Countess']


In [0]:
X_train = X_train.filter(selected_variables)
X_test = X_test.filter(selected_variables)

In [57]:
# Random Search Cross Validation
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,3, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [ 2,3, 4,5]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 3, 5, 10], 'min_samples_leaf': [2, 3, 4, 5], 'bootstrap': [True, False]}


In [58]:
# Use the random grid to search for best hyperparameters
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf,param_distributions = random_grid,n_iter = 100,cv = 5,verbose=2,random_state=42,n_jobs = -1)
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   54.9s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 10.4min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [59]:
rf_best_params = RandomForestClassifier(**rf_random.best_params_)
rf_best_params.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=80, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [60]:
y_predict = rf_best_params.predict(X_test)
cmx = confusion_matrix(y_test, y_predict)
print(cmx)
print(classification_report(y_test, y_predict))

[[86 17]
 [23 53]]
              precision    recall  f1-score   support

           0       0.79      0.83      0.81       103
           1       0.76      0.70      0.73        76

    accuracy                           0.78       179
   macro avg       0.77      0.77      0.77       179
weighted avg       0.78      0.78      0.78       179



In [0]:
y_pred1 = rf_best_params.predict(y.filter(selected_variables))


In [0]:
submission_df = pd.DataFrame(columns=['PassengerId', 'Survived'])
submission_df['PassengerId'] = test_df['PassengerId']
submission_df['Survived'] = y_pred1
submission_df.to_csv('submissions.csv', header=True, index=False)

In [0]:
test_df['y_pred1']=y_pred1

In [0]:
test_df.to_csv('test_df.csv', header=True, index=False)